In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '' 

In [2]:
import malaya_speech
from malaya_speech import Pipeline
import soundfile as sf
from glob import glob
from datasets import Audio
from tqdm import tqdm
import torchaudio
import torch

`pyaudio` is not available, `malaya_speech.streaming.pyaudio` is not able to use.


In [3]:
mp3s = sorted(glob('*.mp3'))
mp3s

['(Audiobook) Mengenali Imam Mahdi dan Akhir Zaman - Bahagian 1 [XpSalODXHKA].mp3',
 '(Audiobook) Mengenali Imam Mahdi dan Akhir Zaman - Bahagian 2 [WS5lY8mZi_4].mp3',
 '(Audiobook) Mengenali Imam Mahdi dan Akhir Zaman - Bahagian 3 [yyFu6xJN8cw].mp3',
 'BUKAN KERANA AKU TAK CINTA (BAB 1) - AUDIOBOOK [hW7F4xT9ZY0].mp3',
 'BUKAN KERANA AKU TAK CINTA (BAB 2) - AUDIOBOOK [rZqXBfimaHU].mp3',
 'BUKAN KERANA AKU TAK CINTA (BAB 3) - AUDIOBOOK [uAgp1LOXf3o].mp3',
 'BUKAN KERANA AKU TAK CINTA (BAB 4) - AUDIOBOOK [u9ulw_KXsdA].mp3',
 'BUKAN KERANA AKU TAK CINTA (BAB 5) - AUDIOBOOK [5r0friFAF3E].mp3',
 'BUKAN KERANA AKU TAK CINTA (BAB 6) - AUDIOBOOK [tUnwczpH0aw].mp3',
 'HARRY POTTER DAN BILIK RAHSIA (BAB 1) ｜ MALAY AUDIOBOOK [1tnfiH-dLDI].mp3',
 'HARRY POTTER DAN BILIK RAHSIA (BAB 2) ｜ MALAY AUDIOBOOK [6PbzdKnLmZA].mp3',
 'HARRY POTTER DAN BILIK RAHSIA (BAB 3) - MALAY AUDIOBOOK [ikQilu3cYsY].mp3',
 'HARRY POTTER DAN BILIK RAHSIA (BAB 4) - MALAY AUDIOBOOK [SVeFRVjO_NM].mp3',
 'HARRY POTTER DAN BIL

In [4]:
p = Pipeline()

model_v2 = malaya_speech.vad.deep_model(model = 'vggvox-v2')
pipeline = (
    p.map(malaya_speech.utils.generator.frames, frame_duration_ms = 30)
    .batching(20)
    .foreach_map(model_v2.predict)
    .flatten()
)

2023-03-05 13:12:29.397769: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-05 13:12:29.426133: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-03-05 13:12:29.426162: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: husein-MS-7D31
2023-03-05 13:12:29.426171: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: husein-MS-7D31
2023-03-05 13:12:29.426474: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 470.161.3
2023-03-05 13:12:29.426489: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported vers

In [5]:
sample_rate = 44100

audio = Audio(sampling_rate=sample_rate)

In [6]:
!rm */*.wav

rm: cannot remove '*/*.wav': No such file or directory


In [7]:
for f in tqdm(mp3s):
    directory_name = f.replace('.mp3', '_').replace('|', '')
    os.makedirs(directory_name, exist_ok=True)
    
    samples = audio.decode_example(audio.encode_example(f))['array']
    # samples = samples[: 60 * sample_rate]
    
    samples_16k = malaya_speech.resample(samples, sample_rate, 16000)
    frames_16k = list(
        malaya_speech.utils.generator.frames(samples_16k, 30, 16000, append_ending_trail = False)
    )
    frames = list(
        malaya_speech.utils.generator.frames(samples, 30, sample_rate, append_ending_trail = False)
    )
    
    result = p.emit(samples_16k)
    frames_deep_v2_batch = [
        (frame, result['flatten'][no]) for no, frame in enumerate(frames)
    ]
    results = malaya_speech.split.split_vad(
        frames_deep_v2_batch, n = 5, negative_threshold = 0.1
    )

    for no in range(len(results)):
        result = results[no]
        filename = f'{no}.mp3'
        torchaudio.save(os.path.join(directory_name, filename), 
                            torch.tensor(result.array.astype('float32')).unsqueeze(0), 
                            sample_rate, format='mp3')
#         sf.write(
#             os.path.join(directory_name, filename),
#             result.array,
#             sample_rate,
#         )

  0%|                                                    | 0/19 [00:00<?, ?it/s]/home/husein/.local/lib/python3.8/site-packages/librosa/core/spectrum.py:222: UserWarning: n_fft=512 is too small for input signal of length=480
  warnings.warn(
/home/husein/.local/lib/python3.8/site-packages/librosa/core/spectrum.py:222: UserWarning: n_fft=512 is too small for input signal of length=303
  warnings.warn(
  5%|██▏                                      | 1/19 [03:22<1:00:49, 202.78s/it]/home/husein/.local/lib/python3.8/site-packages/librosa/core/spectrum.py:222: UserWarning: n_fft=512 is too small for input signal of length=221
  warnings.warn(
 16%|██████▊                                    | 3/19 [08:56<45:32, 170.75s/it]/home/husein/.local/lib/python3.8/site-packages/librosa/core/spectrum.py:222: UserWarning: n_fft=512 is too small for input signal of length=416
  warnings.warn(
 26%|███████████▎                               | 5/19 [14:36<38:18, 164.17s/it]/home/husein/.local/lib/python3.